# 🎯 Thought Anchor 분석 결과 시각화

이 노트북은 `anchor_vectors_output/` 디렉토리의 결과를 시각화합니다.

In [ ]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from IPython.display import display, HTML, Markdown

plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style('whitegrid')

## 📂 결과 파일 로드

In [ ]:
output_dir = Path('anchor_vectors_output')
result_files = sorted(output_dir.glob('example_*.json'))

print(f"총 {len(result_files)}개의 결과 파일 발견:")
for f in result_files:
    size_kb = f.stat().st_size / 1024
    print(f"  - {f.name}: {size_kb:.1f} KB")

## 🔍 개별 예제 상세 분석

In [ ]:
def analyze_example(json_path):
    """단일 예제 분석 및 시각화"""
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    example_name = json_path.stem
    print("="*80)
    print(f"📊 {example_name.upper()}")
    print("="*80)
    
    # 에러 체크
    if 'error' in data:
        print(f"❌ Error: {data['error']}")
        return
    
    print(f"Total pairs: {data['total_pairs']}")
    print(f"Successful pairs: {data['successful_pairs']}")
    print(f"Image: {data.get('image_path', 'N/A')}")
    print()
    
    # QA pair 분석
    for qa_idx, qa_pair in enumerate(data.get('qa_pairs', [])):
        print(f"\n{'='*70}")
        print(f"QA Pair {qa_idx + 1}")
        print(f"{'='*70}\n")
        
        # 질문
        question = qa_pair.get('question', '')
        if '<|vision_start|>' in question:
            question = question.split('\n')[-1]  # 마지막 줄만
        print(f"❓ Question:\n   {question}\n")
        
        # Reasoning
        reasoning = qa_pair.get('reasoning_text', '')
        print(f"💭 Reasoning:\n{reasoning}\n")
        
        # Chunks
        chunks = qa_pair.get('chunks', [])
        anchor_vector = np.array(qa_pair.get('anchor_vector', []))
        
        if len(chunks) == 0:
            print("⚠️  No chunks found.")
            continue
        
        print(f"📝 {len(chunks)} Chunks with Anchor Scores:\n")
        
        # Chunk와 Anchor Score 함께 표시
        for i, (chunk, score) in enumerate(zip(chunks, anchor_vector)):
            # 중요도에 따라 색상 표시
            if score > 12:
                marker = "🔴"  # 매우 중요
            elif score > 10:
                marker = "🟠"  # 중요
            elif score > 5:
                marker = "🟡"  # 보통
            else:
                marker = "⚪"  # 낮음
            
            print(f"  {marker} Chunk {i} (score: {score:.2f}):")
            print(f"     {chunk[:120]}..." if len(chunk) > 120 else f"     {chunk}")
            print()
        
        # Anchor Score 시각화
        if len(anchor_vector) > 1 and anchor_vector.max() > 0:
            fig, ax = plt.subplots(figsize=(10, 4))
            
            colors = ['red' if s > 12 else 'orange' if s > 10 else 'yellow' if s > 5 else 'lightgray' 
                      for s in anchor_vector]
            
            bars = ax.bar(range(len(anchor_vector)), anchor_vector, color=colors, alpha=0.7, edgecolor='black')
            ax.set_xlabel('Chunk Index', fontsize=12)
            ax.set_ylabel('Anchor Score', fontsize=12)
            ax.set_title(f'{example_name} - QA Pair {qa_idx + 1}: Anchor Scores', fontsize=14, fontweight='bold')
            ax.set_xticks(range(len(anchor_vector)))
            ax.grid(axis='y', alpha=0.3)
            
            # 값 표시
            for i, (bar, val) in enumerate(zip(bars, anchor_vector)):
                if val > 0.1:
                    ax.text(bar.get_x() + bar.get_width()/2, val + 0.5, 
                           f'{val:.1f}', ha='center', va='bottom', fontsize=10)
            
            plt.tight_layout()
            plt.show()
            
            # Top 3 앵커 출력
            top_indices = np.argsort(anchor_vector)[-3:][::-1]
            print(f"\n⭐ Top 3 Most Important Anchors:\n")
            for rank, idx in enumerate(top_indices, 1):
                if anchor_vector[idx] > 0:
                    print(f"  {rank}. Chunk {idx} (score: {anchor_vector[idx]:.2f})")
                    print(f"     {chunks[idx][:100]}..." if len(chunks[idx]) > 100 else f"     {chunks[idx]}")
                    print()
        else:
            print("⚠️  Anchor scores are all zero or only one chunk.")

# 예제 선택 (example_1.json 분석)
if result_files:
    # 가장 큰 파일 (성공한 예제일 가능성 높음)
    largest_file = max(result_files, key=lambda f: f.stat().st_size)
    print(f"\n분석할 파일: {largest_file.name}\n")
    analyze_example(largest_file)

## 📊 전체 통계

In [ ]:
# 모든 결과 파일 통계
stats = {
    'total_files': 0,
    'successful': 0,
    'errors': 0,
    'total_qa_pairs': 0,
    'all_anchor_scores': [],
    'chunks_counts': []
}

for json_file in result_files:
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    stats['total_files'] += 1
    
    if 'error' in data:
        stats['errors'] += 1
    elif data.get('successful_pairs', 0) > 0:
        stats['successful'] += 1
        stats['total_qa_pairs'] += data['successful_pairs']
        
        for qa in data.get('qa_pairs', []):
            anchor_vec = qa.get('anchor_vector', [])
            if anchor_vec and len(anchor_vec) > 0:
                stats['all_anchor_scores'].extend(anchor_vec)
                stats['chunks_counts'].append(len(qa.get('chunks', [])))

print(f"📈 전체 통계:")
print(f"  - 총 파일: {stats['total_files']}")
print(f"  - 성공: {stats['successful']}")
print(f"  - 에러: {stats['errors']}")
print(f"  - 총 QA pairs: {stats['total_qa_pairs']}")

if stats['all_anchor_scores']:
    anchor_scores = np.array(stats['all_anchor_scores'])
    non_zero = anchor_scores[anchor_scores > 0]
    
    print(f"\n📊 Anchor Score 분포:")
    print(f"  - 총 점수 개수: {len(anchor_scores)}")
    print(f"  - Non-zero 점수: {len(non_zero)}")
    if len(non_zero) > 0:
        print(f"  - 평균: {non_zero.mean():.2f}")
        print(f"  - 최소: {non_zero.min():.2f}")
        print(f"  - 최대: {non_zero.max():.2f}")
        print(f"  - 표준편차: {non_zero.std():.2f}")
    
    # Anchor Score 분포 히스토그램
    if len(non_zero) > 0:
        plt.figure(figsize=(10, 5))
        plt.hist(non_zero, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
        plt.axvline(non_zero.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {non_zero.mean():.2f}')
        plt.xlabel('Anchor Score', fontsize=12)
        plt.ylabel('Frequency', fontsize=12)
        plt.title('Distribution of Non-Zero Anchor Scores', fontsize=14, fontweight='bold')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

if stats['chunks_counts']:
    print(f"\n📝 Chunks 통계:")
    print(f"  - 평균 chunks 수: {np.mean(stats['chunks_counts']):.1f}")
    print(f"  - 최소 chunks: {min(stats['chunks_counts'])}")
    print(f"  - 최대 chunks: {max(stats['chunks_counts'])}")

## 🔍 특정 예제 선택해서 분석

In [ ]:
# 원하는 예제 번호 선택 (0, 1, 2, ...)
example_number = 1

example_file = output_dir / f'example_{example_number}.json'
if example_file.exists():
    analyze_example(example_file)
else:
    print(f"❌ {example_file.name} not found!")

## 🎨 모든 예제 비교

In [ ]:
# 모든 성공한 예제의 평균 anchor score 비교
example_stats = []

for json_file in result_files:
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if 'error' not in data and data.get('successful_pairs', 0) > 0:
        for qa_idx, qa in enumerate(data.get('qa_pairs', [])):
            anchor_vec = np.array(qa.get('anchor_vector', []))
            if len(anchor_vec) > 0:
                non_zero = anchor_vec[anchor_vec > 0]
                if len(non_zero) > 0:
                    example_stats.append({
                        'name': f"{json_file.stem}_qa{qa_idx}",
                        'max_score': anchor_vec.max(),
                        'mean_score': non_zero.mean(),
                        'num_chunks': len(anchor_vec)
                    })

if example_stats:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    names = [s['name'] for s in example_stats]
    max_scores = [s['max_score'] for s in example_stats]
    mean_scores = [s['mean_score'] for s in example_stats]
    
    # Max scores
    ax1.bar(range(len(names)), max_scores, color='coral', alpha=0.7, edgecolor='black')
    ax1.set_xlabel('Example', fontsize=12)
    ax1.set_ylabel('Max Anchor Score', fontsize=12)
    ax1.set_title('Maximum Anchor Score per Example', fontsize=14, fontweight='bold')
    ax1.set_xticks(range(len(names)))
    ax1.set_xticklabels(names, rotation=45, ha='right')
    ax1.grid(axis='y', alpha=0.3)
    
    # Mean scores
    ax2.bar(range(len(names)), mean_scores, color='skyblue', alpha=0.7, edgecolor='black')
    ax2.set_xlabel('Example', fontsize=12)
    ax2.set_ylabel('Mean Anchor Score', fontsize=12)
    ax2.set_title('Mean Anchor Score per Example', fontsize=14, fontweight='bold')
    ax2.set_xticks(range(len(names)))
    ax2.set_xticklabels(names, rotation=45, ha='right')
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  No successful examples with anchor scores found.")

## 💡 인사이트 요약

In [ ]:
print("🎯 주요 발견사항:\n")
print("1. ✅ Method 3 (Attention Suppression)이 성공적으로 작동")
print("   - KL divergence 계산 정상")
print("   - Thought anchor 식별 가능\n")

print("2. 📊 중요한 앵커는 주로:")
print("   - 계산 과정이 포함된 문장")
print("   - 최종 답변 문장")
print("   - 구체적인 수치/논리가 있는 문장\n")

print("3. ⚠️  개선이 필요한 부분:")
print("   - 일부 예제는 이미지 경로 문제로 스킵됨")
print("   - 너무 짧은 reasoning은 chunk 분할 어려움")
print("   - 더 많은 chartqa 데이터로 테스트 필요")